In [141]:
import geopandas as gpd
import pandas as pd
import numpy as np

# User group 2

In [142]:
#read in the cleaned pedestrian data
pedestrian_network = gpd.read_file('../data/cleaned-pednetwork/cleaned-pednetwork-full/cleaned-pednetwork-full.shp')

# Sidewalks

## 1. Waytype

In [143]:
pedestrian_network['fclass'].unique()

array(['crossing', 'footway', 'steps', 'pedestrian', 'path', 'track',
       'service', 'm_footway', 'residential', 'm_service', 'cycleway',
       'tertiary', 'secondary', 'trunk', 'primary', '0'], dtype=object)

In [144]:
#create a mapping of the fclass to waytype weight
waytype_weight_dict = {
    'service': 1.5,
    'm_service': 1.5,
    'residential': 1.5,
    'tertiary':3,
    'secondary': 3,
    'primary': 3,
    'trunk': 3,
    'cycleway': 2,
}


In [145]:
pedestrian_network['waytype_w'] = pedestrian_network['fclass'].map(waytype_weight_dict).fillna(1).astype(float)

## 2. Sidewalk width

In [146]:
# read in the sidewalk width data
sidewalk_width = pd.read_csv('../data/pednetwork-attributes/sidewalks/paving-and-width.csv')

In [184]:
# pedestrian_network

In [148]:
#join on pednet_id
pedestrian_network = pedestrian_network.merge(sidewalk_width, on='pednet_pid', how='left')

In [149]:
def calculate_width_w(width):
    if pd.isna(width):
        return 1
    elif width < 36:
        return 5
    elif 36 <= width < 48:
        return 2
    elif width >= 48:
        return 1

In [150]:
pedestrian_network['width_w'] = pedestrian_network['width'].apply(calculate_width_w)

## 3. Sidewalk surface type

In [151]:
pedestrian_network['surface_t'].unique()

array(['concrete', 'gravel', 'asphalt', 'brick'], dtype=object)

In [152]:
#craete a mapping of the surface type to surface weight
surface_weight_dict = {
    'concrete': 1,
    'asphalt': 2,
    'brick': 3,
    'gravel': 5,
}

In [153]:
#map the surface type to the surface weight
pedestrian_network['surface_w'] = pedestrian_network['surface_t'].map(surface_weight_dict).fillna(1).astype(float)

## 4. Number of steps

In [154]:
# read in the number of steps data
number_of_steps = pd.read_csv('../data/pednetwork-attributes/sidewalks/number-of-steps.csv')

In [155]:
#join on pednet_id
pedestrian_network = pedestrian_network.merge(number_of_steps, on='pednet_id', how='left')

In [185]:
# pedestrian_network

In [157]:
def calculate_steps_w(steps):
    if pd.isna(steps):
        return 1
    elif 1 <= steps < 5: 
        return 3
    elif steps >= 5:
        return 5

In [158]:
#want to convert to steps_w Nan is 1, < 5 is 2, >=5 < 10 is 3, >=10 is 5
pedestrian_network['steps_w'] = pedestrian_network['number-of-steps'].apply(calculate_steps_w).astype(float)

## 5. Slope

In [160]:
# create a new column for the length of the pedestrian network
pedestrian_network['length'] = pedestrian_network['geometry'].length

In [161]:
# add a column for z difference
pedestrian_network['z_d'] = abs(pedestrian_network['z-start'] - pedestrian_network['z-end'])
#change the z_d to meters to feet
pedestrian_network['z_d'] = pedestrian_network['z_d'] * 3.28084

In [162]:
# create a new column for the length of the pedestrian network
pedestrian_network['length'] = pedestrian_network['geometry'].length

In [163]:
#calculate the slope in percent
pedestrian_network['slope'] = pedestrian_network['z_d'] / pedestrian_network['length']

In [164]:
# add in a slope weight, if slope > 5% then 4, else 1
pedestrian_network['slope_w'] = np.where(pedestrian_network['slope'] > 0.05, 4, 1)

In [165]:
#pedestrian_network.explore(column = 'slope', tiles = 'cartodbpositron', scheme = 'naturalbreaks')

# Crossings

In [166]:
#read in crossing traffic flow data
crossing_traffic_flow = pd.read_csv('../data/pednetwork-attributes/crosswalks/crossings-traffic-flow.csv')

In [167]:
# join on pednet_id
pedestrian_network = pedestrian_network.merge(crossing_traffic_flow, on='pednet_id', how='left')

In [186]:
# pedestrian_network

In [169]:
def calculate_crossing_w(row):
    if row['crossing_t'] == 'm':
        if row['traffic_flow'] == 'low':
            return 2
        elif row['traffic_flow'] == 'mid':
            return 3
        elif row['traffic_flow'] == 'high':
            return 4
    if row['crossing_t'] == 'u':
        if row['traffic_flow'] == 'low':
            return 2
        elif row['traffic_flow'] == 'mid':
            return 4
        elif row['traffic_flow'] == 'high':
            return 999
    return 1  

In [170]:
pedestrian_network['crossing_w'] = pedestrian_network.apply(calculate_crossing_w, axis=1).astype(float)

# Calculate length based on weight


In [171]:
#create a new column for the total weight that is the product of all the weights
pedestrian_network['total_w'] = pedestrian_network['waytype_w'] * pedestrian_network['width_w'] * pedestrian_network['surface_w'] * pedestrian_network['steps_w'] * pedestrian_network['slope_w'] * pedestrian_network['crossing_w']
# if the total weight is greater than 999, set it to 999
pedestrian_network['total_w'] = np.where(pedestrian_network['total_w'] > 999, 999, pedestrian_network['total_w'])

In [172]:
#create a column for ['weighted_length'] that length * total_w
pedestrian_network['w_length'] = pedestrian_network['length'] * pedestrian_network['total_w']

In [173]:
# new column that calculates the ratio change of the weighted length
pedestrian_network['w_length_r'] = pedestrian_network['w_length']/pedestrian_network['length']

In [174]:
#new column that calculates the absolute difference between the weighted length and the length
pedestrian_network['w_length_d'] = pedestrian_network['w_length'] - pedestrian_network['length']

In [187]:
# pedestrian_network.head()

In [176]:
pedestrian_network = pedestrian_network[['pednet_id', 'waytype_w', 'width_w', 'surface_w', 'steps_w', 'slope_w', 'crossing_w', 'length', 'w_length', 'w_length_r', 'w_length_d', 'z-start' ,'z-end','geometry']]

Visual Checks

In [188]:
# pedestrian_network.explore(column = 'w_length_r', tiles = 'cartodbpositron', scheme = 'naturalbreaks')

In [178]:
# pedestrian_network['slope_w'].value_counts()

Saving Data

In [181]:
#save the data to shapefile
pedestrian_network.to_file('../data/weighted-network/user-group-2/user-group-2-weighted-network.shp')

In [183]:
#save also as geojson
pedestrian_network.to_file('../data/paired-data/ug2/network_ug2.geojson', driver='GeoJSON')